In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import time
import numpy as np
import tempfile
import zipfile
import os
from tensorflow_model_optimization.sparsity import keras as sparsity

In [63]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

if tf.keras.backend.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes)

4423680/4422102 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Load pruned model

In [109]:
pruned_model_path = './models/fashion_mnist/fashion_mnist_pruned_sparcity-90.h5'
#final_model = tf.keras.models.load_model(pruned_model_path)

In [110]:
with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(pruned_model_path)

In [111]:
final_model = sparsity.strip_pruning(restored_model)
final_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
flatten_6 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               802944    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)               

In [112]:
final_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [121]:
start = time.time()

score = final_model.evaluate(x_test, y_test, verbose=0)

exec_time = time.time() - start

In [122]:
print('images_count:', x_test.shape[0])
print('exec_time:', exec_time)
print('exec_time_per_image:', exec_time/x_test.shape[0])
print('Test loss:', score[0])
print('Test accuracy:', score[1])

images_count: 10000
exec_time: 3.4114933013916016
exec_time_per_image: 0.00034114933013916016
Test loss: 1.5668896862030028
Test accuracy: 0.895
